In [1]:
# On terminal: conda activate python38

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import ipynb.fs.defs.functions as fct
import pickle

import warnings
warnings.filterwarnings("error")

In [2]:
# Load IDs
with open('uniqueIDs.pkl', 'rb') as f:
    uniqueIDs = pickle.load(f)

# Fit mod1 for each participant

In [3]:
# Model settings

# Functions
value_fct = fct.rescorla_wagner
dec_fct = fct.my_softmax

# Store everything
mod_info = {}
mod_info['name'] = 'model1'
mod_info['value_fct'] = value_fct.__name__
mod_info['dec_fct'] = dec_fct.__name__
mod_info['param_names'] = ['v0', 'alpha', 'beta']
#print(mod['value_fct'].__name__)

# save
all_users_folder = 'data/all_users/mod1/'
file_name = all_users_folder+'mod_parameters.pkl'
with open(file_name, 'wb') as f:
    pickle.dump(mod_info, f)

In [4]:
run_ = True

# Folder
all_users_folder = 'data/all_users/mod1/'
    
if run_:
    
    # Parameter range for initial guess 
    # ['v0', 'alpha', 'beta']
    param_lower_bound = [-10, 0, 0]
    param_upper_bound = [ 10, 1, 20]

    # Fit
    all_users = {}
    p_hit_per_trial = pd.DataFrame([])
    ev_per_trial = pd.DataFrame([])
    for n_part,ID in enumerate(uniqueIDs): 

        # Get data
        user_folder = 'data/user_' + ID + '/'
        df2_cf = pd.read_pickle(user_folder + 'df2_cf.pkl')
        isHit_all_cues, fbs_all_cues, trialNo_all_cues = fct.extract_hits_fbs(df2_cf)

        # Create a new Model object
        mod = fct.Model(mod_name = mod_info['name'],
                     value_fct = value_fct, 
                     dec_fct = dec_fct, 
                     param_names = mod_info['param_names'])

        # Input data to model
        mod.set_data(ID, fbs_all_cues, isHit_all_cues, trialNo_all_cues)

        # Fit model
        mod.fit(param_lower_bound, param_upper_bound, n_iterations=5)

        # Nested dictionnary user data
        all_users[n_part] = {}
        all_users[n_part]['ID']=mod.ID
        all_users[n_part]['nLL']=mod.nLL
        all_users[n_part]['Ntrials']=mod.Ntrials
        all_users[n_part]['Nparams']=len(mod.param_names)
        for i in range(0,len(mod.param_names)):
            all_users[n_part][mod.param_names[i]]=mod.param_values[i]
        
        # Concatenated model predictions: p hit
        tmp = pd.DataFrame(mod.p_hit).transpose()
        tmp.columns = tmp.columns+1
        tmp = tmp.reset_index().rename(columns={'index': 'Cue'})
        tmp.insert(0,'ID',ID)
        p_hit_per_trial = pd.concat([tmp, p_hit_per_trial], axis=0)
        
        # Concatenated model predictions: EVs
        tmp = pd.DataFrame(mod.v).transpose()
        tmp.columns = tmp.columns+1
        tmp = tmp.reset_index().rename(columns={'index': 'Cue'})
        tmp.insert(0,'ID',ID)
        ev_per_trial = pd.concat([tmp, ev_per_trial], axis=0)
    
    # Save mod LLs and parameter values
    mod_fit = pd.DataFrame(all_users).transpose()
    mod_fit.to_pickle(all_users_folder+'mod_param_fits.pkl')
    
    # Save mod predictions
    p_hit_per_trial = p_hit_per_trial.sort_values(by='ID').reset_index(drop=True)
    p_hit_per_trial.to_pickle(all_users_folder+'mod_p_hit_per_trial.pkl')
    ev_per_trial = ev_per_trial.sort_values(by='ID').reset_index(drop=True)
    ev_per_trial.to_pickle(all_users_folder+'mod_ev_per_trial.pkl')
    

In [5]:
mod.v

{'HR': array([-0.8448679 ,  1.59918942,  3.02125338, -0.33286914,  1.89709364,
        -0.98695652,  1.51651578,  0.4642236 ,  0.68826087,  0.81861596,
         2.56708016,  3.58441633,  1.6674224 ,  3.06095444,  2.19915724,
        -0.81120214, -0.8901488 , -0.8901488 , -0.8901488 , -0.93608356,
        -0.93608356, -0.93608356, -0.93608356, -0.93608356, -0.93608356,
        -0.93608356, -0.93608356, -0.93608356]),
 'LR': array([-0.8448679 , -0.90973708, -0.90973708, -0.90973708, -0.90973708,
        -0.90973708, -0.90973708, -0.90973708, -0.90973708, -0.90973708,
        -0.90973708, -0.90973708, -0.90973708, -0.90973708, -0.90973708,
        -0.11117209,  2.02608709,  3.2696419 ,  2.32058112,  1.76837429,
         0.61076636,  0.77352613, -1.64069931, -0.53647924, -0.53647924,
        -0.73030249, -2.51569535,  0.62702584]),
 'HP': array([-0.8448679 , -2.58235474, -2.58235474, -2.58235474, -2.58235474,
        -2.58235474, -2.58235474, -2.58235474, -2.58235474, -2.58235474,
        

In [6]:
mod.PEs

{'HR': array([ 5.8448679 ,  3.40081058, -8.02125338,  5.33286914, -6.89709364,
         5.98695652, -2.51651578,  0.5357764 ,  0.31173913,  4.18138404,
         2.43291984, -4.58441633,  3.3325776 , -2.06095444, -7.19915724,
        -0.18879786,         nan,         nan, -0.1098512 ,         nan,
                nan,         nan,         nan,         nan,         nan,
                nan,         nan,         nan]),
 'LR': array([-0.1551321 ,         nan,         nan,         nan,         nan,
                nan,         nan,         nan,         nan,         nan,
                nan,         nan,         nan,         nan,  1.90973708,
         5.11117209,  2.97391291, -2.2696419 , -1.32058112, -2.76837429,
         0.38923364, -5.77352613,  2.64069931,         nan, -0.46352076,
        -4.26969751,  7.51569535,         nan]),
 'HP': array([-4.1551321 ,         nan,         nan,         nan,         nan,
                nan,         nan,         nan,         nan,         nan,
        

In [16]:
mod_fit

,ID,nLL,Ntrials,Nparams,v0,alpha,beta
0,001,52.836721,112,3,-0.069151,0.002608,19.500275
1,003,53.461215,112,3,0.140338,0.030766,2.847292
2,006,54.597408,112,3,0.564683,0.195588,0.979577
3,007,50.411542,112,3,0.626474,0.041909,1.98518
4,008,44.469531,112,3,0.054125,0.005225,15.224536
...,...,...,...,...,...,...,...
165,453,51.765799,112,3,8.057981,0.294006,0.789725
166,462,44.662409,112,3,0.028651,0.012171,7.72589
167,469,4.867525,112,3,0.02503,0.03969,20.0
168,470,50.395574,112,3,0.05433,0.009292,7.879051


In [50]:
# Create a new Model object
mod1 = fct.Model(mod_name = 'mod1',
                 value_fct = fct.rescorla_wagner, 
                 dec_fct = fct.my_softmax, 
                 param_names = ['v0', 'alpha', 'beta'])

# Input data to model
mod1.set_data(ID, fbs_all_cues, isHit_all_cues, trialNo_all_cues)

In [52]:
# Specifiy model parameters
v0 = -0.066746
alpha = 0.002524
beta = 20

# Run model
nLL = mod1.compute_nLL([v0, alpha, beta])

nLL

52.82896128665941

In [42]:
# Specifiy model parameters
v0 = -0.135
alpha = 0.00499
beta = 12

# Run model
nLL = mod1.compute_nLL([v0, alpha, beta])

nLL

53.51268165204495

In [36]:
# Specifiy model parameters
v0 = -0.135
alpha = 0.00499
beta = 8

# Run model
nLL = mod1.compute_nLL([v0, alpha, beta])

nLL

53.841475328267364